In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Dropout, Flatten, Bidirectional
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

data = pd.read_csv("input.csv")

texts = data['text'].tolist()
classes = data['class'].tolist()

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

tokenized_texts = [word_tokenize(text.lower()) for text in texts]
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

preprocessed_texts = []
for tokens in tokenized_texts:
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token not in punctuation]
    preprocessed_texts.append(filtered_tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
max_sequence_length = 500
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

label_encoder = LabelEncoder()
encoded_classes = label_encoder.fit_transform(classes)
num_classes = len(set(classes))
one_hot_classes = to_categorical(encoded_classes, num_classes=num_classes)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_classes, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)

model2 = Sequential()
model2.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model2.add(LSTM(128))
model2.add(Dropout(0.2))
model2.add(Dense(num_classes, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)

cnn_lstm_predictions = model.predict(X_test)
lstm_rnn_predictions = model2.predict(X_test)

combined_predictions = []
for i in range(len(cnn_lstm_predictions)):
    combined_predictions.append(np.argmax(cnn_lstm_predictions[i] + lstm_rnn_predictions[i]))

combined_accuracy = accuracy_score(np.argmax(y_test, axis=1), combined_predictions)
print("Combined model accuracy:", combined_accuracy)

print("Classification Report:")
print(classification_report(np.argmax(y_test, axis=1), combined_predictions))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Epoch 1/6
48/48 [==============================] - 30s 271ms/step - loss: 1.6432 - accuracy: 0.3784 - val_loss: 0.7797 - val_accuracy: 0.7232
Epoch 2/6
48/48 [==============================] - 12s 254ms/step - loss: 0.4283 - accuracy: 0.8593 - val_loss: 0.4362 - val_accuracy: 0.8401
Epoch 3/6
48/48 [==============================] - 11s 234ms/step - loss: 0.1394 - accuracy: 0.9624 - val_loss: 0.4175 - val_accuracy: 0.8616
Epoch 4/6
48/48 [==============================] - 12s 242ms/step - loss: 0.0533 - accuracy: 0.9871 - val_loss: 0.4695 - val_accuracy: 0.8525
Epoch 5/6
48/48 [==============================] - 10s 200ms/step - loss: 0.0256 - accuracy: 0.9941 - val_loss: 0.4482 - val_accuracy: 0.8695
Epoch 6/6
48/48 [==============================] - 11s 224ms/step - loss: 0.0191 - accuracy: 0.9966 - val_loss: 0.7453 - val_accuracy: 0.8277
Epoch 1/5
48/48 [==============================] - 18s 340ms/step - loss: 1.8880 - accuracy: 0.3120 - val_loss: 1.3706 - val_accuracy: 0.4785
Epoch 

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Dropout, Flatten, Bidirectional
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Step 1: Load the data from the CSV file

data = pd.read_csv("/Users/charansaisadla/Downloads/input-2.csv")

texts = data['text'].tolist()
classes = data['class'].tolist()

# Step 2: Convert the text data into numerical representations
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

tokenized_texts = []
for text in texts:
    tokens = [lemmatizer.lemmatize(token.lower()) for token in word_tokenize(text) if
              token.lower() not in stop_words and token.lower() not in string.punctuation]
    tokenized_texts.append(tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(tokenized_texts)
max_sequence_length = 500
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Step 3: Encode the class labels
label_encoder = LabelEncoder()
encoded_classes = label_encoder.fit_transform(classes)
num_classes = len(label_encoder.classes_)
one_hot_classes = to_categorical(encoded_classes, num_classes=num_classes)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_classes, test_size=0.2, random_state=42)

# Step 5: Define and train the CRNN model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(Conv1D(256, 8, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)

# Step 6: Evaluate the model on the test set
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)
y_test_encoded = y_test.argmax(axis=1)
report = classification_report(y_test_encoded, y_pred)
print(report)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/5


2023-05-09 12:55:35.644279: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


37/37 [==============================] - 28s 745ms/step - loss: 1.8806 - accuracy: 0.3483 - val_loss: 1.1191 - val_accuracy: 0.6053
Epoch 2/5
37/37 [==============================] - 29s 794ms/step - loss: 0.8174 - accuracy: 0.7130 - val_loss: 0.7992 - val_accuracy: 0.7221
Epoch 3/5
37/37 [==============================] - 29s 785ms/step - loss: 0.4261 - accuracy: 0.8592 - val_loss: 0.7758 - val_accuracy: 0.7361
Epoch 4/5
37/37 [==============================] - 29s 785ms/step - loss: 0.1786 - accuracy: 0.9478 - val_loss: 0.8523 - val_accuracy: 0.7327
Epoch 5/5
74/74 [==============================] - 3s 41ms/step
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       214
           1       0.50      0.78      0.61       209
           2       1.00      1.00      1.00       217
           3       0.74      0.37      0.49       213
           4       0.55      0.46      0.50       217
           5       0.49      0.60      0.54       222